# **Feature Selection** 

Feature selection methods are a set of techniques that allow us to choose among the pool of features available. Unlike dimensionality reduction, these retain the features as they are which makes them highly interpretable. They usually belong to one of these three categories:

![image](images/feature_selection.png)


## **Filter Methods** 

Filter methods are the simplest type of feature selection method. They work by filtering features prior to model building based on some criteria. They are computationally inexpensive, since they do not involve testing the subsetted features using a model. It is more difficult to take multivariate relationships into account because we are not evaluating model performance. For example, a variable might not have much predictive power on its own, but can be informative when combined with other variables.

These are statistical techniques used to “filter” out useful features. Filter methods are completely model agnostic (meaning they can be used with any model) and are useful sanity checks for a data scientist to carry out before deciding on a model. They include and are not limited to: correlation coefficients (Pearson, Spearman, etc), $\chi^2$, ANOVA and Mutual Information calculations.


### **Variance Threshold**

One of the most basic filter methods is to use a variance threshold to remove any features that have little to no variation in their values. This is because features with low variance do not contribute much information to a model. Since variance can only be calculated on numeric values, this method only works on quantitative features. That said, we may also want to remove categorical features for which all or a majority of the values are the same. To do that, we would need to dummy code the categorical variables first. Now, we’ll be able to use the `VarianceThreshold` class from `scikit-learn` to help remove the low-variance features. By default, it drops all features with zero variance, but we can adjust the threshold during class instantiation using the `threshold` parameter if we want to allow some variation. The `.fit_transform()` method returns the filtered features as a numpy array:

```python
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0)  # 0 is default

print(selector.fit_transform(X_num))
```

Luckily, `VarianceThreshold` offers another method called `.get_support()` that can return the indices of the selected features, which we can use to manually subset our numeric features DataFrame.

Specify `indices=True` to get indices of selected features:

```python
print(selector.get_support(indices=True))
```

Use indices to get the corresponding column names of selected features:

```python
num_cols = list(X_num.columns[selector.get_support(indices=True)])

print(num_cols)
```

Finally, to obtain our entire features DataFrame, including the categorical column `edu_goal`, we could do:

```python
X = X[['edu_goal'] + num_cols]

print(X)
```

### **Pearson’s Correlation**


![image](images/pearson_correlation.png)


Another type of filter method involves finding the correlation between variables. In particular, the Pearson’s correlation coefficient is useful for measuring the linear relationship between two numeric, continuous variables — a coefficient close to 1 represents a positive correlation, -1 represents a negative correlation, and 0 represents no correlation. Like variance, Pearson’s correlation coefficient cannot be calculated for categorical variables. Although, there is a related point biserial correlation coefficient that can be computed when one variable is dichotomous, but we won’t focus on that here.

Correlation is best suited to continuous, normally distributed data and is thus easily swayed by extreme values. As such, correlation will misrepresent relationships that are not linear. This occurs VERY OFTEN in practice since much of our world is nonlinear and not normally distributed (your stats class notwithstanding). There are 2 main ways of using correlation for feature selection — to detect correlation between features and to detect correlation between a feature and the target variable.

#### **Correlation between Features**

When two features are highly correlated with one another, then keeping just one to be used in the model will be enough because otherwise they provide duplicate information. The second variable would only be redundant and serve to contribute unnecessary noise.

To determine which variables are correlated with one another, we can use the `.corr()` method from `pandas` to find the correlation coefficient between each pair of numeric features in a DataFrame. By default, `.corr()` computes the Pearson’s correlation coefficient, but alternative methods can be specified using the method parameter. We can visualize the resulting correlation matrix using a `heatmap`:

```python
import seaborn as sns

corr_matrix = X_num.corr(method='pearson')  # 'pearson' is default

sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r')
```

Let’s define high correlation as having a coefficient of greater than 0.7 or less than -0.7. We can loop through the correlation matrix to identify the highly correlated variables:

```python
# Loop over bottom diagonal of correlation matrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):

        # Print variables with high correlation
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            print(corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j])
```

 Because they provide redundant information, we can choose to remove one of those variables. To decide which one, we can look at their correlation with the target variable, then remove the one that is less associated with the target. This is explored in the next section.

#### **Correlation between Feature and Target**

As mentioned, the second way correlation can be used is to determine if there is a relationship between a feature and the target variable. In the case of Pearson’s correlation, this is especially useful if we intend to fit a linear model, which assumes a linear relationship between the target and predictor variables. If a feature is not very correlated with the target variable, such as having a coefficient of between -0.3 and 0.3, then it may not be very predictive and can potentially be filtered out.

We can use the same `.corr()` method seen previously to obtain the correlation between the target variable and the rest of the features. 

```python
X_y = X_num.copy()
X_y['exam_score'] = y

corr_matrix = X_y.corr()

# Isolate the column corresponding to `exam_score`
corr_target = corr_matrix[['exam_score']].drop(labels=['exam_score'])
plt.figure(figsize=(3, 10))
sns.heatmap(corr_target, annot=True, fmt='.3', cmap='RdBu_r')
```

#### **F-Statistic**

To conclude this section, we’ll briefly note an alternative approach for assessing the correlation between variables. Instead of generating the full correlation matrix, we could use the `f_regression()` function from scikit-learn to find the F-statistic for a model with each predictor on its own. The F-statistic will be larger (and p-value will be smaller) for predictors that are more highly correlated with the target variable, thus it will perform the same filtering:

```python
from sklearn.feature_selection import f_regression

print(f_regression(X_num, y))
```

The function returns the F-statistic in the first array and the p-value in the second. As seen, the result is consistent with what we had observed in the correlation matrix — the stronger the correlation (either positive or negative) between the feature and target, the higher the corresponding F-statistic and lower the p-value. 

### **Mutual Information**

The final filter method we’ll look at is using mutual information to rank and select the top features. Despite being the goto measure for association in practice Pearson’s correlation is not a general measure of dependence. We say that the information given by a correlation coefficient is not enough to define the dependence structure between random variables. The fact that correlation only looks for linear dependence means it cannot suggest there is no general correlation when it measures 0 (a correlation of 0 does not mean variables are independent). Only in situations where things are simple (e.g. linear) will Pearson’s correlation offer some potential insight into causality (although there are issues with this idea as well). It relies on covariance, which suggests that greater values of one variable mainly correspond with greater values of another variable. While this sounds obvious, it is a simplistic notion of things being related. Variables can be related in all kinds of ways.

To tease out the dependence structure between variables requires we do more than just compare vectors of data. We need to somehow tap directly into the idea of information. After all, that is what we’re doing when we make a measurement. The outstanding question in any measurement is how much information is being shared between the 2 or more things. Mutual information, as its name suggests, looks to find how much information is shared between 2 variables rather than just noting their commensurate “movement.”

Mutual information is a measure of dependence between **two variables** and can be used to gauge how much a feature contributes to the prediction of the target variable. It is similar to Pearson’s correlation, but is not limited to detecting linear associations. This makes mutual information useful for more flexible models where a linear functional form is not assumed. Another advantage of mutual information is that it also works on discrete features or target, unlike correlation. Although, categorical variables need to be numerically encoded first.

![image](images/mutual_information.png)

For two random variables X and Y, the mutual information is defined as:

$I(X;Y)=\Sigma\Sigma P(x,y) \log(\frac{P(x,y)}{P(x)P(y)})$

- If MI is high → X and Y share a lot of information (strong dependency).
- If MI is low (close to 0) → X and Y are nearly independent.
- If MI is 0 → X and Y are completely independent.

#### **mutual_info_regression**

Now, we can compute the mutual information between each feature and the target variable using `mutual_info_regression()`. This function is used because our target variable is continuous, but if we had a discrete target variable, we would use `mutual_info_classif()`. We specify the `random_state` in the function in order obtain reproducible results:

```python
from sklearn.feature_selection import mutual_info_regression

print(mutual_info_regression(X_enc, y, random_state=68))
```

```python
from sklearn.feature_selection import mutual_info_classif
import numpy as np

X = np.array([[1, 0], [0, 1], [1, 1], [0, 0]])
y = np.array([1, 0, 1, 0])

mi_scores = mutual_info_classif(X, y)
```

The estimated mutual information between each feature and the target is returned in a numpy array, where each value is a non-negative number — the higher the value, the more predictive power is assumed.

In order to properly calculate the mutual information, we need to tell `mutual_info_regression()` which features are discrete by providing their index positions using the discrete_features parameter:

```python
print(mutual_info_regression(X_enc, y, discrete_features=[0], random_state=68))
```

Compared to the earlier results, we now get greater mutual information between the feature variable and the target variable once it is correctly interpreted as a discrete feature.

#### **SelectKBest**

Finally, let’s look at using the `SelectKBest` class from scikit-learn to help pick out the top k features with the highest ranked scores. In our case, we are looking to select features that share the most mutual information with the target variable. When we instantiate `SelectKBest`, we’ll specify which scoring function to use and how many top features to select. Here, our scoring function is `mutual_info_regression()`, but because we want to specify additional arguments besides the X and y inputs, we’ll need the help of the `partial()` function from Python’s built-in `functools` module. Then, the `.fit_transform()` method will return the filtered features as a numpy array:

```python
from sklearn.feature_selection import SelectKBest
from functools import partial

score_func = partial(mutual_info_regression, discrete_features=[0], random_state=68)

# Select top 3 features with the most mutual information
selection = SelectKBest(score_func=score_func, k=3)

print(selection.fit_transform(X_enc, y))
```

Like `VarianceThreshold`, `SelectKBest` also offers the `.get_support()` method that returns the indices of the selected features, so we could subset our original features DataFrame:

```python
X = X[X.columns[selection.get_support(indices=True)]]

print(X)
```

### **Information Value (IV)** 

Information Value (IV) is a statistical measure used to quantify the predictive power of an independent variable (feature) in relation to a binary target variable (e.g., default or no default). It evaluates how well a feature differentiates between two groups, typically "good" (non-defaulting) and "bad" (defaulting) customers in credit risk modeling. IV is commonly used to assess the usefulness of variables when building models like logistic regression.

Variables with high IV values are more predictive and valuable for the model. The IV for a feature is calculated as:

$\text{IV}=\Sigma(\text{WoE}_i\times(\text{Good\%}_i-\text{Bad\%}_i))$

There is no one definitive way to categorize IV values, but here is a commonly used categorization:
- IV < 0.02: not useful for prediction
- 0.02 ≤ IV < 0.1: weak predictor
- 0.1 ≤ IV < 0.3: moderate predictor
- 0.3 ≤ IV < 0.5: strong predictor
- IV ≥ 0.5: suspicious predictor

Variables with IV > 0.1 are typically shortlisted for modeling. For categorical variables, IV is calculated by grouping categories and applying the same process. IV helps prioritize which features to retain in logistic regression or other interpretable models.

Unlike correlation, which captures linear relationships, IV considers the non-linear relationship between a feature and the target. Compared to tree-based feature importance (e.g., in XGBoost), IV is more interpretable and suitable for transparent models like logistic regression.



#### **Overfitting**

In addition to feature selection, IV can also be used to identify features that may be causing overfitting in a model. Overfitting occurs when a model is too complex and captures noise in the data rather than the underlying patterns. Features with high IV values may be overfitting the model and should be carefully examined.

A feature with a high IV value indicates that it has a strong predictive power on the target variable in the training data. However, if this feature does not generalize well to the testing data, it could be a sign of overfitting. In other words, if a feature has a high IV value in the training data but a low IV value in the testing data, it may be overfitting the training data and not generalizing well to new, unseen data.

On the other hand, a feature with a moderate IV value that is consistent across both the training and testing data may be a more reliable predictor that is less likely to overfit the training data.

Therefore, it is important to evaluate the IV values of features not only on the training data but also on the testing data to ensure that the selected features are not overfitting. If a feature has a high IV value in the training data but a low IV value in the testing data, it may be wise to consider removing that feature from the model.

#### **Binning**

Another use case for IV is in variable binning, which is the process of grouping continuous variables into discrete categories. IV can be used to determine the optimal number of bins for a given feature. Features with high IV values may benefit from a larger number of bins, while features with low IV values may require fewer bins.

Here are the steps to use IV in variable binning:

1. Divide the continuous variable into a number of bins using a binning method, such as equal frequency or equal width.
2. Calculate the IV value for each bin. This can be done using the same formula as for calculating the IV value of a single feature, but instead of calculating it for the whole feature, we calculate it for each bin.
3. Identify bins with high IV values. Bins with high IV values have a strong predictive power on the target variable, and can be used as predictors in the predictive model.
4. Merge adjacent bins with similar IV values. Bins with similar IV values may have a similar distribution of the target variable, and can be merged to create fewer bins with better predictive power.
5. Evaluate the number and distribution of the resulting bins. The number of bins and their distribution should strike a balance between accuracy and simplicity. Too few bins may oversimplify the data, while too many bins may introduce noise and reduce the model’s predictive power.



#### **Limitations**

Here are a few situations where IV may not be appropriate:

1. Non-linear relationships: IV assumes that the relationship between the feature and the target variable is linear. If there is a non-linear relationship between the feature and the target, then the IV calculation may not be accurate and other methods, such as non-linear regression, may be more appropriate.
2. Imbalanced classes: IV may not be suitable when the target variable has imbalanced classes, where one class has a much smaller proportion of instances than the other. In this case, a more appropriate metric may be area under the receiver operating characteristic curve (AUC-ROC) or precision-recall curve (PRC).
3. High cardinality features: IV may not work well with features that have a large number of unique values, such as text data or categorical features with many categories. In these cases, other methods, such as clustering or dimensionality reduction, may be more effective.
4. Complex feature interactions: IV considers the predictive power of individual features in isolation. In cases where the relationship between features and the target variable is complex and involves interactions between features, other methods, such as decision trees or random forests, may be more appropriate.

### **Feature Importance**

When we fit a supervised machine learning (ML) model, we often want to understand which features are most associated with our outcome of interest. Features that are highly associated with the outcome are considered more “important”. We can use it as a filter method to remove irrelevant features from our model and only retain the ones that are most highly associated with our outcome of interest.

Wrapper methods such as recursive feature elimination use feature importance to more efficiently search the feature space for a model.

Feature importance may also be used for model inspection and communication. For example, stakeholders may be interested in understanding which features are most important for prediction. Feature importance can help us answer this question. There are many different ways to calculate feature importance for different kinds of machine learning models. In this section, we’ll investigate one tree-based method in a little more detail: Gini impurity.


#### **Gini Impurity**

Imagine, for a moment, that you’re interested in building a model to screen candidates for a particular job. In order to build this model, you’ve collected some data about candidates who you’ve hired and rejected in the past. For each of these candidates, suppose that you have data on years of experience and certification status. Which of these features seems to be more important for predicting whether a candidate will be hired? In the first example, we saw that most candidates who had >5 years of experience were hired and most candidates with <5 years were rejected; however, all candidates with certifications were hired and all candidates without them were rejected.

Gini impurity is related to the extent to which observations are well separated based on the outcome variable at each node of the decision tree. For example, in the two trees above, the Gini impurity is higher in the node with all candidates (where there are an equal number of rejected and hired candidates) and lower in the nodes after the split (where most or all of the candidates in each grouping have the same outcome — either hired or rejected).

To estimate feature importance, we can calculate the Gini gain: the amount of Gini impurity that was eliminated at each branch of the decision tree. In this example, certification status has a higher Gini gain and is therefore considered to be more important based on this metric.

To demonstrate how we can estimate feature importance using Gini impurity, we’ll use the breast cancer dataset from `sklearn`. This dataset contains features related to breast tumors. The outcome variable is the diagnosis: either malignant or benign. To start, we’ll load the dataset and split it into a training and test set:


In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

dataset = datasets.load_breast_cancer()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

Next, we’ll fit a decision tree to predict the diagnosis using `sklearn.tree.DecisionTreeClassifier()`. Note that we’re setting `criterion= 'gini'`. This actually tells the function to build the decision tree by splitting each node based on the feature that has the highest Gini gain. By building the tree in this way, we’ll be able to access the Gini importances later.


In [2]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion="gini")

# Fit the decision tree classifier
clf = clf.fit(X_train, y_train)

Next, we can access the feature importances based on Gini impurity as follows:

In [3]:
# Print the feature importances
feature_importances = clf.feature_importances_
feature_importances_df = pd.DataFrame(
    {"feature": X_train.columns, "importance": feature_importances}
)
feature_importances_df.sort_values("importance", ascending=False, inplace=True)
feature_importances_df

,feature,importance
27,worst concave points,0.721064
23,worst area,0.118864
13,area error,0.051515
21,worst texture,0.030901
25,worst compactness,0.013720
5,mean compactness,0.013720
28,worst symmetry,0.011849
29,worst fractal dimension,0.011299
20,worst radius,0.010205
1,mean texture,0.009147


Because Gini impurity is used to train the decision tree itself, it is computationally inexpensive to calculate. However, Gini impurity is somewhat biased toward selecting numerical features (rather than categorical features). It also does not take into account the correlation between features. For example, if two highly correlated features are both equally important for predicting the outcome variable, one of those features may have low Gini-based importance because all of it’s explanatory power was ascribed to the other feature. This issue can be mediated by removing redundant features before fitting the decision tree.

There are many other methods for estimating feature importance beyond calculating Gini gain for a single decision tree.

#### **Aggregate Gini Impurity**

Random forests are an ensemble-based machine learning algorithm that utilize many decision trees (each with a subset of features) to predict the outcome variable. Just as we can calculate Gini importance for a single tree, we can calculate average Gini importance across an entire random forest to get a more robust estimate.



#### **Permutation-based Method**

Another way to test the importance of particular features is to essentially remove them from the model (one at a time) and see how much predictive accuracy suffers. One way to “remove” a feature is to randomly permute the values for that feature, then refit the model. This can be implemented with any machine learning model, including non-tree-based- methods. However, one potential drawback is that it is computationally expensive because it requires us to refit the model many times.

#### **Coefficients**

When we fit a general(ized) linear model (for example, a linear or logistic regression), we estimate coefficients for each predictor. If the original features were standardized, these coefficients can be used to estimate relative feature importance; larger absolute value coefficients are more important. This method is computationally inexpensive because coefficients are calculated when we fit the model. It is also useful for both classification and regression problems (i.e., categorical and continuous outcomes). However, similar to the other methods described above, these coefficients do not take highly correlated features into account.


## **Wrapper Methods**


Machine learning problems often involve datasets with many features. Some of those features might be very important for a specific machine learning model. Other features might be irrelevant. Given a feature set and a model, we would like to be able to distinguish between important and unimportant features (or even important combinations of features). Wrapper methods do exactly that.

A wrapper method for feature selection is an algorithm that selects features by evaluating the performance of a machine learning model on different subsets of features. Wrapper methods search for the best set of features by using a “greedy search strategy”. These algorithms add or remove features one at a time based on how useful those features are to the model.  As this could potentially go on forever, a stopping criterion based on number of features or a model performance metric is typically used. They are computationally expensive because the model needs to be re-fitted for each feature set being tested.

Wrapper methods have some advantages over filter methods. The main advantage is that wrapper methods evaluate features based on their performance with a specific model. Filter methods, on the other hand, can’t tell how important a feature is to a model.

Another upside of wrapper methods is that they can take into account relationships between features. Sometimes certain features aren’t very useful on their own but instead perform well only when combined with other features. Since wrapper methods test subsets of features, they can account for those situations.

This lesson will explain five different wrapper methods:

- Sequential forward selection
- Sequential backward selection
- Sequential forward floating selection
- Sequential backward floating selection
- Recursive feature elimination

### **Sequential Forward Selection**

Sequential forward selection is a wrapper method that builds a feature set by starting with no features and then adding one feature at a time until a desired number of features is reached. In the first step, the algorithm will train and test a model using only one feature at a time. The algorithm keeps the feature that performs best.

In each subsequent step, the algorithm will test the model on each possible new feature addition. Whichever feature improves model performance the most is then added to the feature subset. This process stops once we have the desired number of features.

Let’s say we want to use three features, and we have five features to choose from: age, height, weight, blood_pressure, and resting_heart_rate. Sequential forward selection will train your machine learning model on five different feature subsets: one for each feature.

If the model performs best on the subset {age}, the algorithm will then train and test the model on the following four subsets:

- {age, height}
- {age, weight}
- {age, blood_pressure}
- {age, resting_heart_rate}

If the model performs best on {age, resting_heart_rate}, the algorithm will test the model on the following three subsets:

- {age, height, resting_heart_rate}
- {age, weight, resting_heart_rate}
- {age, blood_pressure, resting_heart_rate}
If the model performs best on {age, weight, resting_heart_rate}, it will stop the algorithm and use that feature set.

Sequential forward selection is a greedy algorithm: instead of checking every possible feature set by brute force, it adds whichever feature gives the best immediate performance gain.

We will use the `SFS` class from Python’s `mlxtend` library to implement sequential forward selection. The parameter `k_features` determines how many features the algorithm will select. `cv` allows you to do k-fold cross-validation. We’ll leave it at 0 for this lesson and only evaluate performance on the training set.

```python
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression

# Logistic regression model
lr = LogisticRegression(max_iter=1000)

# Sequential forward selection
sfs = SFS(lr,
          k_features=3,
          forward=True,
          floating=False,
          scoring='accuracy',
          cv=0)

# Fit the sequential forward selection model
sfs.fit(X, y)
```

The `mlxtend` library also makes it easy to visualize how the accuracy of a model changes as sequential forward selection adds features. You can use the code `plot_sfs(sfs.get_metric_dict())` to create a matplotlib figure that plots the model’s performance as a function of the number of features used.

```python
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import matplotlib.pyplot as plt

# Plot the accuracy of the model as a function of the number of features
plot_sfs(sfs.get_metric_dict())
plt.show()
```

The `.subsets_` attribute allows you to see much of that information, including which feature was chosen at each step and the model’s accuracy after each feature addition. `sfs.subsets_` is a dictionary. The keys in this dictionary are the numbers of features at each step in the sequential forward selection algorithm. The values in the dictionary are dictionaries with information about the feature set at each step. `'avg_score'` is the accuracy of the model with the specified number of features.
 

```python
# Print a tuple of feature names after 5 features are added
print(sfs.subsets_[5]['feature_names'])
```

### **Sequential Backward Selection** 

Sequential backward selection is another wrapper method for feature selection. It is very similar to sequential forward selection, but there is one key difference. Instead of starting with no features and adding one feature at a time, sequential backward selection starts with all of the available features and removes one feature at a time.

To implement sequential backward selection in mlxtend you can use the same `SFS` class you used for sequential forward selection. The only difference is that you have to set the parameter `forward` to `False`.

You can also use `plot_sfs(sfs.get_metric_dict())` to visualize the results of sequential backward selection.

### **Sequential Forward and Backward Floating Selection** 

Sequential forward floating selection is a variation of sequential forward selection. It starts with zero features and adds one feature at a time, just like sequential forward selection, but after each addition, it checks to see if we can improve performance by removing a feature.

- If performance can’t be improved, the floating algorithm will continue to the next step and add another feature.
- If performance can be improved, the algorithm will make the removal that improves performance the most (unless removal of that feature would lead to an infinite loop of adding and removing the same feature over and over again).

Sequential backward floating selection works similarly. Starting with all available features, it removes one feature at a time. After each feature removal, it will check to see if any feature additions will improve performance (but it won’t add any features that would result in an infinite loop).

Floating selection algorithms are sometimes preferable to their non-floating counterparts because they test the model on more possible feature subsets. They can detect useful relationships between variables that plain sequential forward and backward selection might miss.

We can implement sequential forward or backward floating selection in `mlxtend` by setting the parameter `floating` to `True`. The parameter forward determines whether `mlxtend` will use sequential forward floating selection or sequential backward floating selection. 

### **Recursive Feature Elimination**

Recursive feature elimination is another wrapper method for feature selection. It starts by training a model with all available features. It then ranks each feature according to an importance metric and removes the least important feature. The algorithm then trains the model on the smaller feature set, ranks those features, and removes the least important one. The process stops when the desired number of features is reached.

In regression problems, features are ranked by the size of the absolute value of their coefficients.

It’s important to note that you might need to standardize data before doing recursive feature elimination. In regression problems in particular, it’s necessary to standardize data so that the scale of features doesn’t affect the size of the coefficients.

Note that recursive feature elimination is different from sequential backward selection. Sequential backward selection removes features by training a model on a collection of subsets (one for each possible feature removal) and greedily proceeding with whatever subset performs best. Recursive feature elimination, on the other hand, only trains a model on one feature subset before deciding which feature to remove next.

This is one advantage of recursive feature elimination. Since it only needs to train and test a model on one feature subset per feature removal, it can be much faster than the sequential selection methods that we’ve covered.

Once the data is standardized, you can train the model and do recursive feature elimination using `RFE()` from scikit-learn. As before with the sequential feature selection methods, you have to specify a scikit-learn model for the estimator parameter (in this case, lr for our logistic regression model). `n_features_to_select` is self-explanatory: set it to the number of features you want to select.

```python
from sklearn.feature_selection import RFE

# Recursive feature elimination
rfe = RFE(lr, n_features_to_select=2)
rfe.fit(X, y)

#Print feature rankings after recursive feature elimination
print(rfe.ranking_)

## List of features chosen by recursive feature elimination
rfe_features = [f for (f, support) in zip(feature_list, rfe.support_) if support]
print(rfe_features)
```

You can inspect the results of recursive feature elimination by looking at `rfe.ranking_` and `rfe.support_`. `rfe.ranking_` is an array  that contains the rank of each feature.  If you have a list of feature names, you can use a list comprehension and `rfe.support_` to get a list of chosen feature names.

You can use `rfe.score(X, y)` to check the accuracy of the model.

More efficient than brute-force methods. Works well when using tree-based models or linear models with strong coefficients. Computationally expensive when using high-dimensional data. Not useful if feature importance rankings are unstable.

### **Exhaustive Feature Selection**

To perform an exhaustive search, we need to evaluate all possible feature subsets and select the best one based on model performance. This can be done using `mlxtend`'s `ExhaustiveFeatureSelector`. It is however, extremely slow and not feasible for large datasets. It's computational cost increases exponentially with feature count.

```python
from mlxtend.feature_selection import ExhaustiveFeatureSelector
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

# Load dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model
model = RandomForestClassifier(n_estimators=50, random_state=42)

# Perform exhaustive search
efs = ExhaustiveFeatureSelector(
    model, 
    min_features=1,  # Minimum number of features to consider
    max_features=5,  # Maximum number of features to consider
    scoring='accuracy',  # Metric to optimize
    cv=3  # Cross-validation folds
)

# Fit selector on training data
efs.fit(X_train, y_train)

# Get best feature subset
print("Best Feature Indices:", efs.best_idx_)
print("Best Feature Names:", efs.best_feature_names_)
print("Best Accuracy Score:", efs.best_score_)
```


## **Embedded Methods**

Embedded methods also involve building and evaluating models for different feature subsets, but their feature selection process happens at the same time as their model fitting step. Like wrapper methods, they can optimize the feature set for a particular model and account for multivariate relationships. They are also generally less computationally expensive because feature selection happens during model training.

Embedded methods are implemented during the model implementation step. Regularization techniques such as Lasso or Ridge tweak the model to get it to generalize better to new and unknown data. Tree-based feature importance is another widely used embedded method. This method provides insight into the features that are most relevant to the outcome variable while fitting decision trees to the data.

### **Regularization**

Regularization plays a very important role in real world implementations of machine learning models. It is a statistical technique that minimizes overfitting and is executed during the model fitting step. It is also an embedded feature selection method because it is implemented while the parameters of the model are being calculated.

Machine learning algorithms are rarely deployed to production without using some form of regularization. The reason for this is as follows: In practice, every model has to deal with the question of how well it can generalize from known to unknown data. We can train, test and tune models on known data and make them as accurate as possible. However, in deploying models, we are applying them on new data. Regularization makes sure that our model is still accurate.

In this lesson, we are going to learn how regularization minimizes overfitting and how to use it as a feature selection method. Along the way, we are also going to learn two concepts that are very relevant to regularization and important just as standalone topics within machine learning, namely, the bias-variance tradeoff and hyperparameter tuning.

### **Overfitting**

If a model is able to represent a particular set of data points effectively but is not able to fit new data well, it is overfitting the data. Such a model has one or more of the following attributes:

- It fits the training data well but performs significantly worse on test data
- It typically has more parameters than necessary, i.e., it has high model complexity
- It might be fitting for features that are multi-collinear (i.e., features that are highly negatively or positively correlated)
- It might be fitting the noise in the data and likely mistaking the noise for features

In practice, we often catch overfitting by comparing the model performance on the training data versus test data. For instance if the R-squared score is high for training data but the model performs poorly on test data, it’s a strong indicator of overfitting.

### **Penalty Term**

Regularization penalizes models for overfitting by adding a “penalty term” to the loss function. The two most commonly used ways of doing this are known as Lasso (or L1) and Ridge (or L2) regularization.(We’ll explain the meaning behind their names in a moment!)

Both of these rely on penalizing overfitting by controlling how large the coefficients can get in the first place. The penalty term or regularization term is multiplied by a factor alpha and added to the old loss function as follows:

$\text{New loss function} = \text{Old loss function} + \alpha \times \text{Regularisation term}$

Because of the additive term, minimizing the new loss function will necessarily mean “true loss” goes up, i.e., the scores on this will be lower on the training data than regression without regularization! But this is what we want when we are regularizing. Remember that the reason we’re regularizing is because our model is overfitted to our data (i.e., it is performing well on training data but doesn’t generalize well on test data).

The regularization term is the sum of the absolute values of the coefficients in the case of L1 regularization and the sum of the squares of the coefficients in the case of L2.

- L1 Regularisation term $|b_1|+|b_2|$
- L2 Regularisation term $b_1^2+b_2^2$

In mathematics, the sum of the magnitudes of a vector is known as its L1 norm (related to “Manhattan distance”) and the square root of the sum of the magnitudes (or the “Euclidean distance” from the origin) is known as its L2 norm - and that is the reason for the names of both methods

### **L1 Lasso Regularization**

In the case of the two-feature linear regression scenario we were looking at in the previous exercise, our new loss function for L1 regularization looks as follows:

$\text{L1 Loss} = \frac{1}{n}\sum\limits_{i=1}^{n}(y_i-b_0-b_1x_{1i}-b_2x_{2i})^2+\alpha(|b_1|+|b_2|)$ 

Minimizing this new loss function essentially means restricting the size of the regularization term while minimizing the old loss function. Let’s say that our regularization term can take a maximum value, $s$.

This is equivalent to confining our coefficients to a surface around the origin bounded by 4 lines: $b1+b2 = s$, $b1-b2 = s$, $b2-b1 = s$ and $-b1-b2 = s$.

The figure below replicates the elliptical contours for the loss function that we’d plotted earlier in the lesson. If we plot these four lines as with $b_1$ and $b_2$ as X and Y axes respectively, we get the diamond shaped blue shaded region that we’ve shown here. We have chosen a value of $s = 50$ for this figure - this means that either coefficient can have a maximum value of 50. The choice of $s$ is deeply tied to the choice of alpha as they are inversely related.


![images](images/l1_regularisation.png)

The value of $(b_1,b_2)$ that satisfies the regularization constraint while minimizing the loss function is denoted by the white dot. Notice that it is exactly at the tip of the diamond where it intersects with a contour from our loss function. It also happens to fall exactly on the X axis, thus setting the value of $b_2$ to 0!

Why is this the point that minimizes the new loss function? Remember that the goal here is to minimize the original loss function while meeting the regularization constraint. We still want to be as close to the center of the contours (the original loss function minimum) as possible. The point that’s closest to the center of the contours while simultaneously lying within the regularization surface boundary is the white dot!

The word Lasso is actually an acronym for “Least Absolute Shrinkage and Selection Operator” - it shrinks the absolute value of coefficients and selects parameters by way of setting some of the coefficients to zero. If the coefficient of a regression model is set to zero, it means that we’ve pretty much eliminated the feature associated with it. In models with a high number of features, lasso regularization tends to set a significant fraction of its parameters to zero, thus acting as a feature selection method.


```python
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
lasso = Lasso(alpha = 0.1)
lasso.fit(X_train, y_train)

l1_pred_train = lasso.predict(X_train)
l1_mse_train = np.mean((l1_pred_train - y_train)**2)
print("Lasso (L1) Training Error: ", l1_mse_train)

# 2. Calculate testing error
l1_pred_test = lasso.predict(X_test)
l1_mse_test = np.mean((l1_pred_test - y_test)**2)
print("Lasso (L1) Testing Error: ", l1_mse_test)

# 3. Plotting the Coefficients
predictors = X.columns
coef = pd.Series(lasso.coef_,predictors).sort_values()
```

### **L2 Ridge Regularization**

L2 regularization is also called “Ridge” regularization as the mathematical formulation of it belongs to a class of analysis methods known as “ridge analysis”. Similar to L1, this would mean constraining the regularization term to the following surface:

![image](images/l2_regularisation.png)

The key difference here is that instead of a diamond-shaped area, we’re constraining the coefficients to live within a circle of radius $s$ as shown in the figure here. The general goal is still similar though, we want to minimize the old loss while restricting the values of the coefficients to the boundary of this circle. Once again we want our new coefficient values to be as close to the unregularized best fit solution (i.e., the center of the contours) as possible while falling within the circle.

The value of $(b_1,b_2)$ that minimizes this new loss function almost never lies on either axes. The solution here is not the pink dot that lies on the X axis like in L1, but rather the white dot that we have shown. The reason for this is as follows: The circle that contains the white and pink dots represents the smallest value of the old loss function that satisfies the regularization constraint, but while the pink dot makes the regularization term’s value $s^2$ exactly, the white dot makes it even smaller as it lies inside the circle!

Unlike Lasso, our Ridge coefficients can never be exactly zero! L2 regularization is therefore not a feature elimination 
method like L1. The coefficients of L2 get arbitrarily small but never zero. This is particularly useful when we don’t want to get rid of features during modeling but nonetheless want their relative importances emphasized.

```python
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
ridge = Ridge(alpha = 100)
ridge.fit(X_train, y_train)

#Training error
l2_pred_train = ridge.predict(X_train)
l2_mse_train = np.mean((l2_pred_train - y_train)**2)
print("Ridge (L2) Training Error: ", l2_mse_train)

# 2. Calculate testing error
l2_pred_test = ridge.predict(X_test)
l2_mse_test = np.mean((l2_pred_test - y_test)**2)
print("Ridge (L2) Testing Error: ", l2_mse_test)


# 3. Plotting the Coefficients
predictors = X.columns
coef = pd.Series(ridge.coef_,predictors).sort_values()
```

You cannot use Lasso or Ridge from `sklearn.linear_model` directly for classification problems because these models are designed for regression problems.

However, you can use Ridge and Lasso regularization within `LogisticRegression` from `sklearn.linear_model`, as logistic regression is inherently suited for classification tasks.

```python
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Load dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ridge (L2) Regularization
ridge_clf = LogisticRegression(penalty='l2', solver='liblinear', C=1.0)
ridge_clf.fit(X_train, y_train)

# Lasso (L1) Regularization
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=1.0)
lasso_clf.fit(X_train, y_train)

# Model Scores
print("Ridge Accuracy:", ridge_clf.score(X_test, y_test))
print("Lasso Accuracy:", lasso_clf.score(X_test, y_test))
```

Ridge (L2) and Lasso (L1) regularization are specific to linear models like Logistic Regression and Linear Regression. They cannot be directly applied to K-Nearest Neighbors (KNN) or Decision Trees because:

- KNN is a non-parametric model → It does not have coefficients that can be penalized like in Ridge/Lasso.
- Decision Trees (DTs) do not use coefficients → They split data using rules rather than weighting features with coefficients.

### **Alpha Hyperparameter**

Alpha is what is known as a hyperparameter. It is not learned during the model fitting step the way model parameters are. Rather, it’s chosen prior to fitting the model and is used to control the learning process. In regularization, the choice of alpha determines how much we want to control for overfitting during the model fitting step.

We’re able to do this by using alpha to control the size of the constraint surface and consequently the size of the coefficients themselves. The larger the alpha, the smaller the size of the allowed coefficients. In essence, alpha is inversely proportional to $s$ in the case of L1 regularization or $s^2$ in the case of L2 regularization.

When we add the regularization term to our loss function, we are in essence introducing bias into our problem, i.e., we are biasing our model to have coefficients within the regularization boundary. The greater the alpha, the smaller the coefficients and the more biased the model.

### **Bias-Variance Tradeoff**

Recall that the reason we wanted to perform regularization was to prevent our model from overfitting the data. Such a model is said to have high variance as it is likely fitting for random errors or noise within the data. We introduced the bias term to minimize the variance, but if we’re not careful and allow it to get arbitrarily large, we run the risk of underfitting the model!

Ideally we want a machine learning model to have low bias and low variance, i.e., we want it to perform well on training data as well as test data. However, trying to minimize bias and variance simultaneously is a bit of a conundrum as lowering one raises the other! This dilemma in machine learning models is known as the bias-variance tradeoff.

Hyperparameter tuning helps us find a sweet spot in this tradeoff to ensure that neither bias nor variance get too high. Typically, a portion of the data is set aside that is known as the “validation set” or “holdout set” (over and above the usual test-train split) and this is used to perform hyperparameter tuning on.

# **Extra Reading**

https://code.likeagirl.io/this-sklearn-tool-for-feature-selection-is-a-must-know-for-every-ml-and-data-science-professional-4b4a0b86d342

https://medium.com/@shridharpawar77/lasso-regularization-for-feature-selection-a-practical-guide-with-python-51911e712128

